# Homework 4 - Getting to know your customers

In [70]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sympy import *
import time

pd.options.mode.chained_assignment = None

## 1. Finding Similar Costumers

The idea behind the work is to implement *Locality-Sensitive Hashing (LSH)* algorithm to find the most similar users to the query.


We mixed up different approaches taking inspiration mainly from [this site](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/) and the [book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf). The work is composed of the following steps:


<ol>
  <li>Set up the data</li>
  <li>Fingerprint hashing</li>
  <li>Locality Sensitive Hashing</li>
</ol>

### 1.1 Set up the data

For the sake of this first part, not all columns are necessary since comparing each field single handedly can be quite time-expensive. 

We have therefore decided to keep only some columns of the dataset and to create a new one, *CustGroupsBalance*, in order to decrease the number of values ​​to consider and improve the execution time of the algorithm in the next phase.


In [53]:
bank_transactions = pd.read_csv(r"C:\Users\Marina\OneDrive\Desktop\ADM_HW4\bank_transactions.csv")
bank_transactions = bank_transactions.dropna()
bank_transactions = bank_transactions.reset_index(drop=True)

We group the transactions according to their account balance: *lowBalance*, *mediumBalance*, *highBalance*.

In [54]:
groupBalance = []

for balance in bank_transactions['CustAccountBalance']:
    if balance < 5000:
        groupBalance.append('lowBalance')
    elif 5000 <= balance < 50000:
        groupBalance.append('mediumBalance')
    else:
        groupBalance.append('highBalance')
        

bank_transactions['CustGroupsBalance'] = groupBalance

After the pre-processing phase, the dataset with which we will work is presented.

In [55]:
transactions = bank_transactions[['CustomerDOB',  'CustGender', 'CustLocation', 'CustGroupsBalance', 'TransactionAmount (INR)', 'TransactionDate']]
transactions

,CustomerDOB,CustGender,CustLocation,CustGroupsBalance,TransactionAmount (INR),TransactionDate
0,10/1/94,F,JAMSHEDPUR,mediumBalance,25.0,2/8/16
1,4/4/57,M,JHAJJAR,lowBalance,27999.0,2/8/16
2,26/11/96,F,MUMBAI,mediumBalance,459.0,2/8/16
3,14/9/73,F,MUMBAI,highBalance,2060.0,2/8/16
4,24/3/88,F,NAVI MUMBAI,mediumBalance,1762.5,2/8/16
...,...,...,...,...,...,...
1041609,8/4/90,M,NEW DELHI,mediumBalance,799.0,18/9/16
1041610,20/2/92,M,NASHIK,mediumBalance,460.0,18/9/16
1041611,18/5/89,M,HYDERABAD,highBalance,770.0,18/9/16
1041612,30/8/78,M,VISAKHAPATNAM,mediumBalance,1000.0,18/9/16


### 1.2 Fingerprint hashing

We implemented **our MinHash function**: the goal was to replace a large set of values with a smaller *"signature"* that still preserves the underlying similarity metric.

We established the following pipeline:

1. At first, we create the shingles matrix, where a shingle is an individual element that can be a part of a dataset like a character, integer or date.
1. We randomly permute the shingles matrix;
2. For each transaction, start from the top and find the position of the first shingle match and use the shingle indexes to represent the transaction (signature);
3. We repeat it several times and each time we append the result to the signature matrix.

Since permutation is a computation heavy operation especially for large datasets, we use a hashing/mapping function that typically reorders the elements using the simple math operation:

 $$ h(x) = (ax + b)  \% c  $$
 
 where a and b are random integers smaller than c and c is the prime number slightly higher than the total number of different elements in the dataset (that is the number of elements in the shingles matrix).

In [56]:
def permute(x, a, b, c):
    return (a*x + b) % c

In [57]:
def createShinglesMatrix(df):
    shingles_matrix = []

    for column in df:
        for shingle in df[column].unique():
            shingles_matrix.append(shingle)

    diff = nextprime(len(shingles_matrix)) - len(shingles_matrix)
    for _ in range(diff):
        shingles_matrix.append('NA')

    return shingles_matrix

In [58]:
def createListOfPermutations(arr, permutations):
    listOfPermutations = []
    for j in range(30):
        l = {}
        for i,item in enumerate(arr):
            l[item] = permute(i, permutations[j][0], permutations[j][1], len(arr))

        listOfPermutations.append(l)

    return listOfPermutations

In [59]:
def createSignature(df, listOfPermutations, nPermutations):
    signature_matrix=[]

    for row in df.iterrows():
        signature_k =[]
        for j in range(nPermutations):
            index=[listOfPermutations[j][row[1][s]] for s in range(6)]
            signature_k.append(min(index))
        
        signature_matrix.append(signature_k)

    return signature_matrix

In [60]:
# initialization of the shingles matrix
listOfValues = createShinglesMatrix(transactions) 
# computation of random values for "a" and "b" parameters
permutations = [(random.randint(0,len(listOfValues)), random.randint(0,len(listOfValues))) for _ in range(30)] 
# initialization of the list of permutations
listOfPermutations = createListOfPermutations(listOfValues, permutations) 
# computation of the signature
signatureA = createSignature(transactions, listOfPermutations, 10) 

In [71]:
transactions['signature'] = signatureA

In [72]:
transactions

,CustomerDOB,CustGender,CustLocation,CustGroupsBalance,TransactionAmount (INR),TransactionDate,signature,7
0,10/1/94,F,JAMSHEDPUR,mediumBalance,25.0,2/8/16,"[17551, 22446, 19342, 9406, 8235, 5070, 4605, ...",NaN
1,4/4/57,M,JHAJJAR,lowBalance,27999.0,2/8/16,"[3731, 4762, 10173, 11670, 38509, 11104, 46906...",NaN
2,26/11/96,F,MUMBAI,mediumBalance,459.0,2/8/16,"[17551, 15866, 1004, 9406, 37987, 16935, 37045...",NaN
3,14/9/73,F,MUMBAI,highBalance,2060.0,2/8/16,"[17551, 15866, 9581, 9406, 23372, 39185, 12541...",NaN
4,24/3/88,F,NAVI MUMBAI,mediumBalance,1762.5,2/8/16,"[13582, 19584, 412, 9406, 14414, 16935, 2680, ...",NaN
...,...,...,...,...,...,...,...,...
1041609,8/4/90,M,NEW DELHI,mediumBalance,799.0,18/9/16,"[4673, 37647, 31012, 11670, 20721, 16935, 239,...",NaN
1041610,20/2/92,M,NASHIK,mediumBalance,460.0,18/9/16,"[24431, 42339, 31012, 11670, 3773, 16935, 2309...",NaN
1041611,18/5/89,M,HYDERABAD,highBalance,770.0,18/9/16,"[7937, 42339, 28511, 11670, 23372, 27533, 8871...",NaN
1041612,30/8/78,M,VISAKHAPATNAM,mediumBalance,1000.0,18/9/16,"[2704, 25226, 31012, 11670, 38977, 2186, 205, ...",NaN


In [11]:
signatureAdf = pd.DataFrame(signatureA)
trasposed_signatureA = signatureAdf.T.copy(deep = True)
trasposed_signatureA.index = [i for i in range(1,11)]
trasposed_signatureA

,0,1,2,3,4,5,6,7,8,9,...,1041604,1041605,1041606,1041607,1041608,1041609,1041610,1041611,1041612,1041613
1,13488,12771,13488,6367,9956,12054,35571,5650,13488,11337,...,31273,10951,13488,38443,7088,8522,8276,28898,13488,37812
2,12769,27326,12769,12769,12769,7209,12769,27326,10258,27326,...,12769,12769,27326,27326,4594,5029,4968,214,3405,27326
3,27388,27388,8646,15176,27388,27388,27388,27388,9349,15879,...,6344,19145,25822,17226,14973,13159,27888,33715,244,374
4,32426,3782,61070,18104,82015,18104,3782,10405,28035,3782,...,3782,34944,10405,3782,1689,10405,10405,10405,10405,10405
5,356,6630,356,3198,356,3198,3198,9659,356,9659,...,3198,356,356,5271,7833,356,356,17857,356,14911
6,22913,17361,22913,5687,22913,991,22913,12766,22913,17361,...,35733,37703,2773,17361,12376,17361,17361,17361,17361,5955
7,5749,12953,5749,6256,5749,17667,6256,4273,5749,6256,...,22443,2242,5749,33044,11374,5749,5749,9795,5749,33044
8,15558,743,32613,30373,17798,2983,21511,6696,16005,1190,...,23539,49594,28010,33060,18687,1891,25212,30373,22079,30373
9,4936,14706,24476,33041,26247,33041,33041,4980,14750,24520,...,6168,25394,15673,25606,7842,26247,26133,42811,26247,30400
10,6784,33409,6784,6784,6784,6784,6784,32911,3592,39990,...,6784,6784,19791,4090,19791,19791,19791,8103,19791,4090


The signature matrix above is now divided into b bands. In our case, we are setting *b = 2*, which means that we will consider any transactions with the same first two rows to be similar. The larger we make b the less likely there will be another transaction that matches all of the same permutations.

We computed buckets for each band with the *create_buckets(b, trasposed_signature)* function, that will take as input a band b and the transposition of the signature matrix and will return a dict contraining as key the bucket ids and as value a list of the transactions indexes that were mapped to that bucket for the given band b: $$ {bucket_{id}} = [transation1_{idx}, transaction2_{idx},...] $$

In [12]:
def listAlphabet():
  return list(map(chr, range(97, 123)))

In [13]:
def create_bucktes(b, trasposed_signature): 
    buckets = {}
    letters = listAlphabet()

    for i in range(1, len(trasposed_signature), b):
        sub = pd.DataFrame(trasposed_signature.loc[i:i+b-1].copy(deep = True))
        for col in sub:
            arr = sub[col].to_numpy(copy = True)
            
            k = letters[int(i/2 + 1)]
            for j in range(len(arr)):
                k += str(arr[j])

            if k in buckets:
                buckets[k].append(col)

            else:
                buckets[k] = [col]


    return buckets

In [14]:
bucketsA = create_bucktes(2, trasposed_signatureA)

### 1.3 Locality Sensitive Hashing

At first, we pre-process the query dataset as we did for the original dataset.

What we're going to do is compare two pairs of signatures, one from the query and one from the original dataset.

We need to focus our attention only on pairs that are likely to *be similar*, without investigating every pair. There is a general theory of how to provide such focus, called **locality-sensitive hashing (LSH)** (or near-neighbor search).

In [16]:
query = pd.read_csv(r"C:\Users\Marina\OneDrive\Desktop\ADM_HW4\query_users.csv")

In [17]:
groupBalance = []

for balance in query['CustAccountBalance']:
    if balance < 5000:
        groupBalance.append('lowBalance')
    elif 5000 <= balance < 50000:
        groupBalance.append('mediumBalance')
    else:
        groupBalance.append('highBalance')
        

query['CustGroupsBalance'] = groupBalance

In [18]:
final_query = query[['CustomerDOB',  'CustGender', 'CustLocation', 'CustGroupsBalance', 'TransactionAmount (INR)', 'TransactionDate']]


In [74]:
signatureB = createSignature(final_query, listOfPermutations, 10)
final_query['signature'] = signatureB

In [20]:
final_query

,CustomerDOB,CustGender,CustLocation,CustGroupsBalance,TransactionAmount (INR),TransactionDate,signature
0,27/7/78,M,DELHI,highBalance,65.00,2/9/16,"[27773, 27326, 17002, 10405, 52077, 4369, 2061..."
1,6/11/92,M,PANCHKULA,mediumBalance,6025.00,2/9/16,"[13488, 7582, 3056, 10405, 356, 7053, 5749, 41..."
2,14/8/91,M,PATNA,mediumBalance,541.50,10/8/16,"[13488, 19797, 8419, 10405, 356, 4385, 5749, 5..."
3,3/1/87,M,CHENNAI,highBalance,1000.00,29/8/16,"[8051, 22275, 13862, 10405, 9552, 17361, 24768..."
4,4/1/95,M,GURGAON,highBalance,80.00,25/9/16,"[12545, 10258, 12456, 7064, 17574, 17361, 3304..."
5,10/1/81,M,WORLD TRADE CENTRE BANGALORE,mediumBalance,303.00,11/9/16,"[9235, 2492, 22487, 8373, 356, 13983, 5749, 39..."
6,20/9/76,F,CHITTOOR,mediumBalance,20.00,28/8/16,"[13488, 12769, 27888, 2625, 356, 13166, 5749, ..."
7,10/4/91,M,MOHALI,lowBalance,50.00,2/8/16,"[10620, 24815, 838, 3782, 20395, 17361, 17564,..."
8,19/3/90,M,MOHALI,lowBalance,300.00,26/8/16,"[12103, 24815, 32401, 3782, 19879, 17361, 7431..."
9,19/12/70,M,SERAMPORE,highBalance,299.00,27/8/16,"[14093, 9548, 42980, 5136, 6957, 3425, 19721, ..."


In [21]:
signatureBdf = pd.DataFrame(signatureB)
trasposed_signatureB = signatureBdf.T.copy(deep = True)
trasposed_signatureB.index = [i for i in range(1,11)]
trasposed_signatureB

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1,27773,13488,13488,8051,12545,9235,13488,10620,12103,14093,...,14971,2954,8522,8522,14971,32980,13488,25896,13488,13488
2,27326,7582,19797,22275,10258,2492,12769,24815,24815,9548,...,20949,27326,12769,12769,11379,12769,7918,23491,12769,27326
3,17002,3056,8419,13862,12456,22487,27888,838,32401,42980,...,2227,19145,13159,13159,9552,24660,13862,36038,27888,27888
4,10405,10405,10405,10405,7064,8373,2625,3782,3782,5136,...,3782,10405,1689,3043,10405,1028,10405,10405,7398,10405
5,52077,356,356,9552,17574,356,356,20395,19879,6957,...,18569,356,3198,3198,18569,3198,356,42518,356,356
6,4369,7053,4385,17361,17361,13983,13166,17361,17361,3425,...,17361,17361,20307,30059,16144,4124,17361,17361,36240,4369
7,20612,5749,5749,24768,33044,5749,5749,17564,7431,19721,...,24768,2313,25943,28562,18071,4170,5749,28303,5749,5749
8,30373,41945,5154,30373,19723,39582,27951,45188,43796,8793,...,29616,46159,18245,17935,29614,21572,9052,20617,1837,20170
9,21511,26247,26247,40014,42811,26247,347,2004,2004,19887,...,36017,25394,7842,33041,277,33041,11844,26203,17763,26247
10,32911,7280,6843,20942,32465,12944,6784,43182,43182,368,...,20942,8591,6784,6784,32911,6784,20942,32911,4090,12712


In [22]:
bucketsB = create_bucktes(2, trasposed_signatureB)

With the *find_neighbors(bucketsA, bucketsB)* function we are going to take as input two buckets (structured like dictionaries), respectively the one related to the original dataset and the one related to the query, and we will return for each element of the query which elements of the original dataset ended up in the same bucket as the latter.

In [24]:
def find_neighbors(bucketsA, bucketsB):
    neighbors = {}
    for k,v in bucketsB.items():
        if k in bucketsA:
            for elemA in v:
                for elemB in bucketsA[k]:
                    if elemA in neighbors:
                        neighbors[elemA].append(elemB)
                    else:
                        neighbors[elemA] = [elemB]
    return neighbors

With the *check_neighbors(dfA, dfB, neighbors, threshold)* function we take as input the original dataframe and that of the query, a list of all the neighbors for each element of the query computed as described above and we check how much their signatures match computing their similarity through the *jaccard_similarity(signA, signB)* function. We will return a dictionary, *similars*, in which the keys are the query indexes and the values the ids of the costumers who actually respect the threshold value.

Those dissimilar pairs that do map to the same bucket are false positives; we hope these will be only a small fraction of all pairs.

In [46]:
def jaccard_similarity(signA, signB):
    return sum([1 for a, b in zip(signA, signB) if a == b]) / len(signA)

In [25]:
def check_neighbors(dfA, dfB, neighbors, threshold):
    similars = {}
    for k,v in neighbors.items():
        signB = dfB.loc[k]['signature']
        for elem in v:
            signA = dfA.loc[elem]['signature']
            sim = jaccard_similarity(signA, signB)
            if sim >= threshold:
                if k in similars:
                    similars[k].append(bank_transactions.loc[elem]['CustomerID'])
                else:
                    similars[k] = [bank_transactions.loc[elem]['CustomerID']]

    return similars

In [26]:
neighbors = find_neighbors(bucketsA, bucketsB)

In [49]:
start_time = time.time()
similars = check_neighbors(transactions, final_query, neighbors, 0.5)
end_time = time.time()
print("LSH Similarity computed in: " + str((end_time - start_time)//60) + " minutes." )


LSH Similarity computed in: 11.0 minutes.


In [73]:
final_query['similars'] = np.nan
for k,v in similars.items():
    final_query['similars'].loc[k] = sorted(v)

In [31]:
final_query

,CustomerDOB,CustGender,CustLocation,CustGroupsBalance,TransactionAmount (INR),TransactionDate,signature,similars
0,27/7/78,M,DELHI,highBalance,65.00,2/9/16,"[27773, 27326, 17002, 10405, 52077, 4369, 2061...","[C1014370, C1018226, C1019523, C1021572, C1030..."
1,6/11/92,M,PANCHKULA,mediumBalance,6025.00,2/9/16,"[13488, 7582, 3056, 10405, 356, 7053, 5749, 41...","[C1012265, C1014228, C1015334, C1017162, C1023..."
2,14/8/91,M,PATNA,mediumBalance,541.50,10/8/16,"[13488, 19797, 8419, 10405, 356, 4385, 5749, 5...","[C1011124, C1011124, C1011127, C1011127, C1013..."
3,3/1/87,M,CHENNAI,highBalance,1000.00,29/8/16,"[8051, 22275, 13862, 10405, 9552, 17361, 24768...","[C1011140, C1011159, C1011171, C1013425, C1013..."
4,4/1/95,M,GURGAON,highBalance,80.00,25/9/16,"[12545, 10258, 12456, 7064, 17574, 17361, 3304...","[C1038384, C1138321, C1158630, C1234065, C1312..."
5,10/1/81,M,WORLD TRADE CENTRE BANGALORE,mediumBalance,303.00,11/9/16,"[9235, 2492, 22487, 8373, 356, 13983, 5749, 39...","[C1012629, C1013912, C1018636, C1019344, C1019..."
6,20/9/76,F,CHITTOOR,mediumBalance,20.00,28/8/16,"[13488, 12769, 27888, 2625, 356, 13166, 5749, ...","[C1010014, C1010060, C1010113, C1010241, C1010..."
7,10/4/91,M,MOHALI,lowBalance,50.00,2/8/16,"[10620, 24815, 838, 3782, 20395, 17361, 17564,...","[C1010342, C1011912, C1011944, C1016516, C1019..."
8,19/3/90,M,MOHALI,lowBalance,300.00,26/8/16,"[12103, 24815, 32401, 3782, 19879, 17361, 7431...","[C1010342, C1010342, C1022861, C1089230, C1094..."
9,19/12/70,M,SERAMPORE,highBalance,299.00,27/8/16,"[14093, 9548, 42980, 5136, 6957, 3425, 19721, ...","[C1414814, C1414814, C2914879, C2914879, C2914..."


In [32]:
query.loc[0]

CustomerDOB                    27/7/78
CustGender                           M
CustLocation                     DELHI
CustAccountBalance            94695.61
TransactionDate                 2/9/16
TransactionTime                 140310
TransactionAmount (INR)           65.0
CustGroupsBalance          highBalance
Name: 0, dtype: object

In [44]:
bank_transactions.loc[bank_transactions.CustomerID == 'C1019523']

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustGroupsBalance
228195,T229826,C1019523,18/4/77,M,DELHI,65461.17,7/8/16,161727,1000.0,highBalance


We can conclude that hashing method works properly with various thresholds, but the computation times are very long and exceed 10 minutes.

## Command Line

### 1. Which location has the maximum number of purchases been made?

In [152]:
bank_transactions.groupby(['CustLocation'])['CustLocation'].count().sort_values(ascending=False).head(1)

CustLocation
MUMBAI    101997
Name: CustLocation, dtype: int64

### 2. In the dataset provided, did females spend more than males, or vice versa?

In [153]:
avgFemalesTransactions = bank_transactions.loc[bank_transactions.CustGender == "F"]['TransactionAmount (INR)'].mean()
avgMalesTransactions = bank_transactions.loc[bank_transactions.CustGender == "M"]['TransactionAmount (INR)'].mean()
print("Total average spent by females: ", avgFemalesTransactions)
print("Total average spent by males: ", avgMalesTransactions)

Total average spent by females:  1643.9584570348936
Total average spent by males:  1537.3411848042588


### 3. Report the customer with the highest average transaction amount in the dataset.

In [154]:
bank_transactions.groupby('CustomerID')['TransactionAmount (INR)'].mean().sort_values(ascending=False).head(1)

CustomerID
C7319271    1560034.99
Name: TransactionAmount (INR), dtype: float64